### Spark Aggregate Functions

#### Funciones simples de agregacion

In [0]:
%run "../includes/configuration"

In [0]:
movies_df = spark.read.parquet(f"{silver_folder_path}/movies")

In [0]:
from pyspark.sql.functions import count, countDistinct, sum

In [0]:
movies_df.select(count("*")).show()

In [0]:
# Cuando hacemos conteo por una columna especifica, solo cuenta los valores que no son nulos
movies_df.select(count("year_release_date")).show()

In [0]:
# Cuenta los distintos valores, fechas sin repetir
movies_df.select(countDistinct("year_release_date")).show()

In [0]:
movies_df.select(sum("budget")).display()

In [0]:
# Usamos combinacion de funciones de agregacion (sum, count) y filtramos por year_release_date == 2016 y renombramos columnas
movies_df.filter("year_release_date == 2016") \
        .select(sum("budget"), count("movie_id")) \
        .withColumnRenamed("sum(budget)", "total_budget") \
        .withColumnRenamed("count(movie_id)", "count_movies") \
        .display()

### Group By

In [0]:
from pyspark.sql.functions import count, countDistinct, sum, max, min, avg

In [0]:
# Agrupamos por la columna year_release_date y apartir de la agrupacion de esta columna, va hacer una sumatoria correspondiente a cada agrupacion

# Entonces nos devuelve la sumatoria correspondiente a la agrupacion por año
movies_df \
    .groupBy("year_release_date") \
        .sum("budget") \
        .display()

In [0]:
# Como haria para saber cual es el maximo presupuesto por año?

# 1 - Cuando nosotros hacemos una agregacion por una columna, unicamente podemos usar una funcion de agregacion.
# 2 - Si utilizamos mas de una funcion de agregacion nos devuelve un error max is not supported.
# 3 - Para usar mas de dos funciones de agregacion, Databricks nos da la posibilidad de usar el metodo agg()

movies_df \
    .groupBy("year_release_date") \
        .sum("budget") \
        .max("budget") \
        .display()

In [0]:
# Usar la funcion agg()
# Agrupamos por la columna year_release_date y apartir de la agrupacion de esta columna, hacemos cada una de las funciones de agregacion,sum, avg, max, min, count

#.alias, podemos cambiar el nombre de las columnas
movie_group_by_df  = movies_df \
    .groupBy("year_release_date") \
        .agg(
            sum("budget").alias("total_budget"),
            avg("budget").alias("avg_budget"),
            max("budget").alias("max_budget"),
            min("budget").alias("min_budget"),
            count("movie_id").alias("count_movies"),
        )

In [0]:
display(movie_group_by_df)

### Window Functions

- Permiten realizar calculos sobre un subconjunto de filas relacionadas con la fila actual sin necesidad de agrupar el conjunto de dato completo como ocurre con las funciones de agregacion standard
- Estas funciones son ideales para tareas como calcular totales acumulados, identificar valores maximos por grupos y agregamar valores entre filas adyacentes mateniendo la granularidad del Data Frame Original

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc, dense_rank

In [0]:
# Estas funciones de ventana tienen el mismo concepto como si estuvieras trabajando con SQL

# Agregar una columna adicional llamada rank, en donde haga un ranking sobre las peliculas pero ordenado de manera descendente a partir del presupuesto (budget)

# dense_rank(), establece un ranking consecutivo, en donde si hay dos o mas peliculas con el mismo presupuesto, se les asigna el mismo rank

movie_rank = Window.partitionBy("year_release_date").orderBy(desc("budget"))

movie_dense_rank = Window.partitionBy("year_release_date").orderBy(desc("budget"))

movies_df.select("title","budget","year_release_date") \
        .filter("year_release_date is not null") \
        .withColumn("rank", rank().over(movie_rank)) \
        .withColumn("dense_rank", dense_rank().over(movie_dense_rank)) \
        .display()

- se requiere obtener informacion sobre el "total de presupuesto" y el "total de ingresos" de las "peliculas" donde el "año de fecha de lanzamiento" debe ser mayor o igual a 2015, tambien debe estar agrupado por el "año de fecha de lanzamiento" y el "genero" al que pertenece cada pelicula.

- Tambien se requiere realizar un ranking de manera ascendente por el "total de presupuesto" y el "total de ingresos" particionado por "año de fecha de lanzamiento"